In [1]:
import random

from distributed_processing.serializers import DummySerializer
from distributed_processing.client import Client
from distributed_processing.filesystem_connector import FileSystemConnector


In [2]:
CURRO = True

if CURRO:
    NS_PATH ="G:\\fs_namespaces\\prueba_distribuida"
    NS_PATH ="C:\\fs_namespaces\\prueba_distribuida"

else:
    NS_PATH = "/home/augusto/python/notebooks/fs_namespaces/prueba_distribuida"


In [3]:
import logging
logging.getLogger("distributed_processing").setLevel(logging.DEBUG)
#logging.getLogger("distributed_processing.filesystem_connector").setLevel(logging.DEBUG)

In [4]:
fs_connector = FileSystemConnector(NS_PATH)
fs_connector.with_watchdog=True
fs_connector.pop_watchdog_timeout = 10

client = Client(DummySerializer(), fs_connector, check_registry="cache")

registry_lock set
registry_lock unset
nclients_lock set
nclients_lock unset
Client with id: fs_client_2
Results queue: fs_client_2_responses
2025-10-29T18:20:36.278005 Client: fs_client_2 with responses queue: fs_client_2_responses connected


In [5]:
fs_connector.methods_registry()

registry_lock set
registry_lock unset


{'add': ['requests_cola_1'],
 'dic': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval'],
 'info': ['requests_fs_server_1', 'requests_fs_server_2'],
 'lista': ['requests_cola_1'],
 'mul': ['requests_cola_1'],
 'sleep': ['requests_cola_1'],
 'tupla': ['requests_cola_1']}

In [6]:
y = client.rpc_async("add", [1, 0])

2025-10-29T18:20:36.338341 Client: fs_client_2 sent request with id: fs_client_2:1 to queue: requests_cola_1


In [7]:
y.get()

2025-10-29T18:20:36.522383 Client: fs_client_2 received a Single RESULT with id: fs_client_2:1


1

In [8]:
client.check_registry ="always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

'method_queues_fake'


In [9]:
client.check_registry ="never" # use queue client.default_queue, by default "default"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


2025-10-29T18:20:36.582125 Client: fs_client_2 sent request with id: fs_client_2:2 to queue: requests_cola_1


2025-10-29T18:20:36.720261 Client: fs_client_2 received a Single ERROR with id: fs_client_2:2


Error -32601 : The method does not exist/is not available.

 


In [10]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

2025-10-29T18:20:36.750501 Client: fs_client_2 sent request with id: fs_client_2:3 to queue: requests_cola_1


2025-10-29T18:20:36.893130 Client: fs_client_2 received a Single ERROR with id: fs_client_2:3


'Esto es un error'

In [11]:
client.check_registry ="cache"

In [12]:
x = client.rpc_async("div", [1, 0])

2025-10-29T18:20:36.960954 Client: fs_client_2 sent request with id: fs_client_2:4 to queue: requests_cola_1


In [13]:
try:
    x.get()
except Exception as e:
    print(e)

2025-10-29T18:20:37.146336 Client: fs_client_2 received a Single ERROR with id: fs_client_2:4


Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "G:\python\distributed_processing\python\distributed_processing\worker.py", line 272, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_26504\1531640066.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [14]:
client.rpc_sync("add", [1, 1])

2025-10-29T18:20:37.183501 Client: fs_client_2 sent request with id: fs_client_2:5 to queue: requests_cola_1


2025-10-29T18:20:37.410718 Client: fs_client_2 received a Single RESULT with id: fs_client_2:5


2

In [15]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

2025-10-29T18:20:37.446444 Client: fs_client_2 sent request with id: fs_client_2:6 to queue: requests_py_eval


In [16]:
y.get()

2025-10-29T18:20:37.733251 Client: fs_client_2 received a Single RESULT with id: fs_client_2:6


3.0

In [17]:
client.rpc_sync_fn(f, [3.0, 2.0])

2025-10-29T18:20:37.762564 Client: fs_client_2 sent request with id: fs_client_2:7 to queue: requests_py_eval
2025-10-29T18:20:37.983004 Client: fs_client_2 received a Single RESULT with id: fs_client_2:7


5.0

In [18]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "lista", "tupla", "dic"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

t = ("sleep", [10], {})
tp.append(t)
fs.append(client.rpc_async(t[0], t[1]))



2025-10-29T18:20:38.025499 Client: fs_client_2 sent request with id: fs_client_2:8 to queue: requests_cola_1
2025-10-29T18:20:38.050576 Client: fs_client_2 sent request with id: fs_client_2:9 to queue: requests_cola_1
2025-10-29T18:20:38.070269 Client: fs_client_2 sent request with id: fs_client_2:10 to queue: requests_cola_1
2025-10-29T18:20:38.089784 Client: fs_client_2 sent request with id: fs_client_2:11 to queue: requests_cola_1
2025-10-29T18:20:38.109729 Client: fs_client_2 sent request with id: fs_client_2:12 to queue: requests_cola_1
2025-10-29T18:20:38.128678 Client: fs_client_2 sent request with id: fs_client_2:13 to queue: requests_cola_1
2025-10-29T18:20:38.147923 Client: fs_client_2 sent request with id: fs_client_2:14 to queue: requests_cola_1
2025-10-29T18:20:38.169724 Client: fs_client_2 sent request with id: fs_client_2:15 to queue: requests_cola_1
2025-10-29T18:20:38.191323 Client: fs_client_2 sent request with id: fs_client_2:16 to queue: requests_cola_1


('add', [0.7501963847343517, 0.4779159087758683], {})
('div', [0.7694940692317184, 0.6375476970069366], {})
('tupla', [0.991218956023185, 0.5780321505727966], {})
('lista', [0.8460453720359106, 0.08624746065361533], {})
('tupla', [0.09558744999947355, 0.26345322142552696], {})
('add', [0.9672942720936755, 0.4658076237828148], {})
('tupla', [0.3470155993807471, 0.9401780947421374], {})
('tupla', [0.7331892738345197, 0.9247304345446324], {})
('add', [0.8065394592808202, 0.47662341644456807], {})


2025-10-29T18:20:38.214415 Client: fs_client_2 sent request with id: fs_client_2:17 to queue: requests_cola_1
2025-10-29T18:20:38.240580 Client: fs_client_2 sent request with id: fs_client_2:18 to queue: requests_cola_1


('dic', [0.005344947280527368, 0.47394148839141714], {})


In [19]:
resultados = [f.get() for f in fs]
resultados

2025-10-29T18:20:38.287650 Client: fs_client_2 received a Single RESULT with id: fs_client_2:8
2025-10-29T18:20:38.288648 Client: fs_client_2 received a Single RESULT with id: fs_client_2:9
2025-10-29T18:20:38.288648 Client: fs_client_2 received a Single RESULT with id: fs_client_2:10
2025-10-29T18:20:38.289645 Client: fs_client_2 received a Single RESULT with id: fs_client_2:11
2025-10-29T18:20:38.299619 Client: fs_client_2 received a Single RESULT with id: fs_client_2:12
2025-10-29T18:20:38.362538 Client: fs_client_2 received a Single RESULT with id: fs_client_2:13
2025-10-29T18:20:38.416433 Client: fs_client_2 received a Single RESULT with id: fs_client_2:14
2025-10-29T18:20:38.501175 Client: fs_client_2 received a Single RESULT with id: fs_client_2:15
2025-10-29T18:20:38.512952 Client: fs_client_2 received a Single RESULT with id: fs_client_2:16
2025-10-29T18:20:38.611267 Client: fs_client_2 received a Single RESULT with id: fs_client_2:17
2025-10-29T18:20:48.660237 Client: fs_clie

[1.22811229351022,
 1.206959217081677,
 (0.991218956023185, 0.5780321505727966),
 [0.8460453720359106, 0.08624746065361533],
 (0.09558744999947355, 0.26345322142552696),
 1.4331018958764903,
 (0.3470155993807471, 0.9401780947421374),
 (0.7331892738345197, 0.9247304345446324),
 1.2831628757253881,
 {'a': 0.005344947280527368, 'b': [0.005344947280527368, 0.47394148839141714]},
 None]

In [20]:
fs = client.rpc_multi_async(tp)

2025-10-29T18:20:48.708843 Client: fs_client_2 sent request with id: fs_client_2:19 to queue: requests_cola_1
2025-10-29T18:20:48.724469 Client: fs_client_2 sent request with id: fs_client_2:20 to queue: requests_cola_1
2025-10-29T18:20:48.739978 Client: fs_client_2 sent request with id: fs_client_2:21 to queue: requests_cola_1
2025-10-29T18:20:48.755606 Client: fs_client_2 sent request with id: fs_client_2:22 to queue: requests_cola_1
2025-10-29T18:20:48.782576 Client: fs_client_2 sent request with id: fs_client_2:23 to queue: requests_cola_1
2025-10-29T18:20:48.819469 Client: fs_client_2 sent request with id: fs_client_2:24 to queue: requests_cola_1
2025-10-29T18:20:48.840129 Client: fs_client_2 sent request with id: fs_client_2:25 to queue: requests_cola_1
2025-10-29T18:20:48.861173 Client: fs_client_2 sent request with id: fs_client_2:26 to queue: requests_cola_1
2025-10-29T18:20:48.885864 Client: fs_client_2 sent request with id: fs_client_2:27 to queue: requests_cola_1
2025-10-29

In [21]:
# AsynResult.status updates the information every time it is called.
# 'PENDING' state should be assumed as transitory. 
# If there are responses available in the queue or in the cache 
# status or pending() updates the AsyncResult object.

[f.status for f in fs]

2025-10-29T18:20:48.979712 Client: fs_client_2 received a Single RESULT with id: fs_client_2:19
2025-10-29T18:20:48.980756 Client: fs_client_2 received a Single RESULT with id: fs_client_2:20
2025-10-29T18:20:48.981755 Client: fs_client_2 received a Single RESULT with id: fs_client_2:21
2025-10-29T18:20:48.981755 Client: fs_client_2 received a Single RESULT with id: fs_client_2:22
2025-10-29T18:20:49.008279 Client: fs_client_2 received a Single RESULT with id: fs_client_2:23


['OK',
 'OK',
 'OK',
 'OK',
 'OK',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING']

In [22]:
client.wait_responses(timeout=2)

2025-10-29T18:20:49.118122 Client: fs_client_2 received a Single RESULT with id: fs_client_2:24
2025-10-29T18:20:49.129504 Client: fs_client_2 received a Single RESULT with id: fs_client_2:25
2025-10-29T18:20:49.140795 Client: fs_client_2 received a Single RESULT with id: fs_client_2:26
2025-10-29T18:20:49.251939 Client: fs_client_2 received a Single RESULT with id: fs_client_2:27
2025-10-29T18:20:49.251939 Client: fs_client_2 received a Single RESULT with id: fs_client_2:28


['fs_client_2:29']

In [23]:
[f.status for f in fs]

['OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'PENDING']

In [24]:
client.wait_responses()

2025-10-29T18:20:59.282816 Client: fs_client_2 received a Single RESULT with id: fs_client_2:29


[]

In [25]:
# AsynResult.status updates information

[f.status for f in fs]

['OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK']

In [26]:
try:
    client.wait_responses(["kk", "qq"])
except ValueError as e:
    print(e)

wait_responses: ['kk', 'qq'] neither in responses nor in pending.


In [27]:
client._update_cache_with_all_available_responses()

In [28]:
client.pending

{}

In [29]:
[f.get() for f in fs]

[1.22811229351022,
 1.206959217081677,
 (0.991218956023185, 0.5780321505727966),
 [0.8460453720359106, 0.08624746065361533],
 (0.09558744999947355, 0.26345322142552696),
 1.4331018958764903,
 (0.3470155993807471, 0.9401780947421374),
 (0.7331892738345197, 0.9247304345446324),
 1.2831628757253881,
 {'a': 0.005344947280527368, 'b': [0.005344947280527368, 0.47394148839141714]},
 None]

In [30]:
fs = client.rpc_batch_async(tp)

2025-10-29T18:20:59.445766 Client: fs_client_2 sent batch request with 11 requests to queue: requests_cola_1


In [31]:
[f.get() for f in fs]

2025-10-29T18:21:09.669179 Client: fs_client_2 received a Batch Response with 11 items
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:30 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:31 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:32 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:33 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:34 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:35 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:36 from BATCH response
2025-10-29T18:21:09.669179 Client: fs_client_2 processed a RESULT with id: fs_client_2:37 from BATCH response
2025-10-29T18:21:09.669179 Client

[1.22811229351022,
 1.206959217081677,
 (0.991218956023185, 0.5780321505727966),
 [0.8460453720359106, 0.08624746065361533],
 (0.09558744999947355, 0.26345322142552696),
 1.4331018958764903,
 (0.3470155993807471, 0.9401780947421374),
 (0.7331892738345197, 0.9247304345446324),
 1.2831628757253881,
 {'a': 0.005344947280527368, 'b': [0.005344947280527368, 0.47394148839141714]},
 None]

In [32]:
client.rpc_batch_sync(tp)

2025-10-29T18:21:09.707010 Client: fs_client_2 sent batch request with 11 requests to queue: requests_cola_1
2025-10-29T18:21:19.873022 Client: fs_client_2 received a Batch Response with 11 items
2025-10-29T18:21:19.873022 Client: fs_client_2 processed a RESULT with id: fs_client_2:41 from BATCH response
2025-10-29T18:21:19.874018 Client: fs_client_2 processed a RESULT with id: fs_client_2:42 from BATCH response
2025-10-29T18:21:19.874018 Client: fs_client_2 processed a RESULT with id: fs_client_2:43 from BATCH response
2025-10-29T18:21:19.875016 Client: fs_client_2 processed a RESULT with id: fs_client_2:44 from BATCH response
2025-10-29T18:21:19.875359 Client: fs_client_2 processed a RESULT with id: fs_client_2:45 from BATCH response
2025-10-29T18:21:19.875992 Client: fs_client_2 processed a RESULT with id: fs_client_2:46 from BATCH response
2025-10-29T18:21:19.875992 Client: fs_client_2 processed a RESULT with id: fs_client_2:47 from BATCH response
2025-10-29T18:21:19.875992 Client:

[1.22811229351022,
 1.206959217081677,
 (0.991218956023185, 0.5780321505727966),
 [0.8460453720359106, 0.08624746065361533],
 (0.09558744999947355, 0.26345322142552696),
 1.4331018958764903,
 (0.3470155993807471, 0.9401780947421374),
 (0.7331892738345197, 0.9247304345446324),
 1.2831628757253881,
 {'a': 0.005344947280527368, 'b': [0.005344947280527368, 0.47394148839141714]},
 None]

In [33]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('add', [0.5671484572122902, 0.6721163119989023], {})
('div', [0.05355220165975705, 0.8227624949133524], {})
('div', [0.8277238008244179, 0.2772032167379438], {})
('fake', [0.780030572192945, 0.16198771875730245], {})
('div', [0.18876522253407602, 0.05612819155782234], {})
('add', [0.9547396584917908, 0.207971526061691], {})
('add', [0.04784060233319831, 0.012250131629590544], {})
('add', [0.9450776816725078, 0.5145295150441241], {})
('mul', [0.8674873158376103, 0.17940742212588912], {})
('div', [0.8147924785198472, 0.1060241123006217], {})


In [34]:
tp

[('add', [0.5671484572122902, 0.6721163119989023], {}),
 ('div', [0.05355220165975705, 0.8227624949133524], {}),
 ('div', [0.8277238008244179, 0.2772032167379438], {}),
 ('fake', [0.780030572192945, 0.16198771875730245], {}),
 ('div', [0.18876522253407602, 0.05612819155782234], {}),
 ('add', [0.9547396584917908, 0.207971526061691], {}),
 ('add', [0.04784060233319831, 0.012250131629590544], {}),
 ('add', [0.9450776816725078, 0.5145295150441241], {}),
 ('mul', [0.8674873158376103, 0.17940742212588912], {}),
 ('div', [0.8147924785198472, 0.1060241123006217], {})]

In [35]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

2025-10-29T18:21:19.972218 Client: fs_client_2 sent batch request with 10 requests to queue: requests_cola_1


In [36]:
[f.safe_get() for f in fs]

2025-10-29T18:21:20.131325 Client: fs_client_2 received a Batch Response with 10 items
2025-10-29T18:21:20.132321 Client: fs_client_2 processed a RESULT with id: fs_client_2:52 from BATCH response
2025-10-29T18:21:20.132321 Client: fs_client_2 processed a RESULT with id: fs_client_2:53 from BATCH response
2025-10-29T18:21:20.133318 Client: fs_client_2 processed a RESULT with id: fs_client_2:54 from BATCH response
2025-10-29T18:21:20.133318 Client: fs_client_2 processed a ERROR with id: fs_client_2:55 from BATCH response
2025-10-29T18:21:20.134315 Client: fs_client_2 processed a RESULT with id: fs_client_2:56 from BATCH response
2025-10-29T18:21:20.134315 Client: fs_client_2 processed a RESULT with id: fs_client_2:57 from BATCH response
2025-10-29T18:21:20.134315 Client: fs_client_2 processed a RESULT with id: fs_client_2:58 from BATCH response
2025-10-29T18:21:20.135312 Client: fs_client_2 processed a RESULT with id: fs_client_2:59 from BATCH response
2025-10-29T18:21:20.135312 Client:

[1.2392647692111924,
 0.06508828731357862,
 2.985981946980482,
 None,
 3.3631089350101937,
 1.1627111845534817,
 0.060090733962788856,
 1.459607196716632,
 0.15563366306133264,
 7.684973359735165]

In [37]:
client.responses

{}

In [38]:
client.check_registry ="cache"

In [39]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [40]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [41]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

2025-10-29T18:21:20.276758 Client: fs_client_2 sent request with id: fs_client_2:62 to queue: requests_cola_1


In [42]:
try:
    x.get()
except Exception as e:
    print(e)

2025-10-29T18:21:20.394923 Client: fs_client_2 received a Single ERROR with id: fs_client_2:62


Error -32601 : The method does not exist/is not available.

 


In [43]:
y = client.rpc_async("add", [1, 0])

2025-10-29T18:21:20.409100 Client: fs_client_2 sent request with id: fs_client_2:63 to queue: requests_cola_1


In [44]:
y.get(5)

2025-10-29T18:21:20.490816 Client: fs_client_2 received a Single RESULT with id: fs_client_2:63


1

In [45]:
def f(x,y): return x + y

client.check_registry = "never" # "never" use queue "default" don't work for rpc_async_fn or rpc_sync_fn 
y = client.rpc_async_fn(f, [1, 2.0])
try:
    print(y.get())
except Exception as e:
    print(e)

2025-10-29T18:21:20.533251 Client: fs_client_2 sent request with id: fs_client_2:64 to queue: requests_cola_1
2025-10-29T18:21:20.651513 Client: fs_client_2 received a Single ERROR with id: fs_client_2:64


Error -32601 : The method does not exist/is not available.

 


In [46]:
client.check_registry = "cache"
y = client.rpc_async_fn(f, [1, 2.0])

2025-10-29T18:21:20.681803 Client: fs_client_2 sent request with id: fs_client_2:65 to queue: requests_py_eval


In [47]:
y.get()

2025-10-29T18:21:20.846113 Client: fs_client_2 received a Single RESULT with id: fs_client_2:65


3.0

In [48]:
[k for k in client.responses.keys()]

[]

In [49]:
client.clean_used()

In [50]:
[k for k in client.responses.keys()]

[]

In [51]:
resultados

[1.22811229351022,
 1.206959217081677,
 (0.991218956023185, 0.5780321505727966),
 [0.8460453720359106, 0.08624746065361533],
 (0.09558744999947355, 0.26345322142552696),
 1.4331018958764903,
 (0.3470155993807471, 0.9401780947421374),
 (0.7331892738345197, 0.9247304345446324),
 1.2831628757253881,
 {'a': 0.005344947280527368, 'b': [0.005344947280527368, 0.47394148839141714]},
 None]

In [52]:
client.rpc_sync_fn(print, ["hola"])

2025-10-29T18:21:21.053247 Client: fs_client_2 sent request with id: fs_client_2:66 to queue: requests_py_eval


2025-10-29T18:21:21.288184 Client: fs_client_2 received a Single RESULT with id: fs_client_2:66


In [84]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("sleep", "sleep"))
    t = (fn, [15], {})
    print(t)
    tp.append(t)

('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})
('sleep', [15], {})


In [85]:
client.check_registry="never"
fs = client.rpc_multi_async(tp, retry=True)

2025-10-29T18:24:36.572522 Client: fs_client_2 sent request with id: fs_client_2:81 to queue: requests_cola_1
2025-10-29T18:24:36.591696 Client: fs_client_2 sent request with id: fs_client_2:82 to queue: requests_cola_1


2025-10-29T18:24:36.608649 Client: fs_client_2 sent request with id: fs_client_2:83 to queue: requests_cola_1
2025-10-29T18:24:36.627235 Client: fs_client_2 sent request with id: fs_client_2:84 to queue: requests_cola_1
2025-10-29T18:24:36.645875 Client: fs_client_2 sent request with id: fs_client_2:85 to queue: requests_cola_1
2025-10-29T18:24:36.663878 Client: fs_client_2 sent request with id: fs_client_2:86 to queue: requests_cola_1
2025-10-29T18:24:36.683535 Client: fs_client_2 sent request with id: fs_client_2:87 to queue: requests_cola_1
2025-10-29T18:24:36.706829 Client: fs_client_2 sent request with id: fs_client_2:88 to queue: requests_cola_1
2025-10-29T18:24:36.727459 Client: fs_client_2 sent request with id: fs_client_2:89 to queue: requests_cola_1
2025-10-29T18:24:36.749196 Client: fs_client_2 sent request with id: fs_client_2:90 to queue: requests_cola_1


In [86]:
from time import time

def gather(arl, timeout=None, step=5, max_dts=0):
    # Update to reset delta times
    pending_ars = [ar for ar in arl if not(ar.ok() or ar.failed())] 
    pending_ids = [ar.id for ar in pending_ars]
    if fs[0]._client.wait_responses(pending_ids, timeout=0) == []:
        return

    N = len(arl)

    max_dts = max_dts

    t_0 = time()

    i = 0
    # ok() and failed() don't update.
    # pending() updates, every time is called, all the ids 
    # that have a response available in the queue 
    pending_ars = [ar for ar in arl if not(ar.ok() or ar.failed())]
    pending_ars0= pending_ars
    pending_ids = [ar.id for ar in pending_ars]
    t_prev = time()
    n_pending_prev = len(pending_ars)
    n_pending_0 = n_pending_prev    
    while (time() - t_0) <= timeout:
        try:
            if fs[0]._client.wait_responses(pending_ids, timeout=5) == []:
                return
        except TimeoutError:
            pass
        dts = [ar.finished_time - t_0 for ar in pending_ars0 if (ar.ok() or ar.failed())]
        if len(dts)>0:
            #max_dts = max(max_dts, max(dts))
            max_dts = max(dts)    
        pending_ars = [ar for ar in arl if not(ar.ok() or ar.failed())] 
        pending_ids = [ar.id for ar in pending_ars]
        if len(pending_ars) < n_pending_prev:
            t_prev = time()
            n_pending_prev = len(pending_ars)  

        avg =  (time()-t_0)/(n_pending_0 -len(pending_ars)) if (n_pending_0 -len(pending_ars))>0 else 0

        print(f"{i}: seconds {time()-t_0}s, AR recovered {N-len(pending_ars)}, \
                AR left {len(pending_ars)}, max delta {max_dts}, avg {avg}")
        i += 0


In [87]:
gather(fs, 30, 5)

0: seconds 5.191293239593506s, AR recovered 0,                 AR left 10, max delta 0, avg 0
0: seconds 10.312388896942139s, AR recovered 0,                 AR left 10, max delta 0, avg 0


2025-10-29T18:24:51.662589 Client: fs_client_2 received a Single RESULT with id: fs_client_2:81
2025-10-29T18:24:51.765059 Client: fs_client_2 received a Single RESULT with id: fs_client_2:82


0: seconds 15.471536874771118s, AR recovered 2,                 AR left 8, max delta 12.585705757141113, avg 7.735768437385559
0: seconds 20.562876224517822s, AR recovered 2,                 AR left 8, max delta 12.585705757141113, avg 10.281438112258911
0: seconds 25.767588138580322s, AR recovered 2,                 AR left 8, max delta 12.585705757141113, avg 12.883794069290161


2025-10-29T18:25:06.732387 Client: fs_client_2 received a Single RESULT with id: fs_client_2:83
2025-10-29T18:25:06.743358 Client: fs_client_2 received a Single RESULT with id: fs_client_2:84


0: seconds 30.941699743270874s, AR recovered 4,                 AR left 6, max delta 27.564005374908447, avg 7.7354249358177185


In [88]:
[f.status for f in fs]

['OK',
 'OK',
 'OK',
 'OK',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING']

In [89]:
[(time() if f.finished_time is None else f.finished_time) -f.creation_time for f in fs]

[15.09006667137146,
 15.172368049621582,
 30.122740030288696,
 30.115127563476562,
 33.5927574634552,
 33.5747549533844,
 33.554741859436035,
 33.53180432319641,
 33.51117563247681,
 33.48943829536438]

In [90]:
[f.finished_time for f in fs]

[1761758691.6625898,
 1761758691.7650595,
 1761758706.7323873,
 1761758706.743359,
 None,
 None,
 None,
 None,
 None,
 None]

In [91]:
client.pending

{'fs_client_2:85': 1761758676.646874,
 'fs_client_2:86': 1761758676.6648765,
 'fs_client_2:87': 1761758676.6848896,
 'fs_client_2:88': 1761758676.707827,
 'fs_client_2:89': 1761758676.7284558,
 'fs_client_2:90': 1761758676.750193}

In [92]:
fs[3].status

'OK'

In [93]:
fs[0].retry()

2025-10-29T18:25:10.336171 Client: Not Retrying: Request with id: fs_client_2:81 is already received.


In [1]:
[f.retry() for f in fs if ok is not None]

NameError: name 'fs' is not defined

In [63]:
import datetime

In [64]:
datetime.datetime.now().isoformat()

'2025-10-29T18:22:36.906351'

In [65]:
from datetime import datetime

In [66]:
client.check_registry="always"
client.all_queues_for_method("info")

['requests_fs_server_1', 'requests_fs_server_2']

In [67]:
client.connector.all_queues_for_method("info")

['requests_fs_server_1', 'requests_fs_server_2']

In [68]:
client.update_registry()

registry_lock set
registry_lock unset


In [69]:
client.check_registry="Never"
client.all_queues_for_method("hola")

['requests_cola_1']

In [70]:
client.check_registry="always"
a = client.rpc_async("info")

2025-10-29T18:22:37.090210 Client: fs_client_2 sent request with id: fs_client_2:77 to queue: requests_fs_server_2


In [71]:
client.rpc_sync("div", [2,1], timeout=10)

2025-10-29T18:22:37.118667 Client: fs_client_2 sent request with id: fs_client_2:78 to queue: requests_cola_1


2025-10-29T18:22:37.260453 Client: fs_client_2 received a Single RESULT with id: fs_client_2:77
2025-10-29T18:22:37.273416 Client: fs_client_2 received a Single RESULT with id: fs_client_2:78


2.0

In [72]:
x = a.get()

In [73]:
x[1]["requests_cola_1"]["add"](1,2)

3

In [74]:
client.registry

{'add': ['requests_cola_1'],
 'dic': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval'],
 'info': ['requests_fs_server_1', 'requests_fs_server_2'],
 'lista': ['requests_cola_1'],
 'mul': ['requests_cola_1'],
 'sleep': ['requests_cola_1'],
 'tupla': ['requests_cola_1']}

In [75]:
def registry_one_step(x):
    worker_id = x[0]
    registry = {}
    for queue, methods in x[1].items():
        for method in methods:
            if method in registry:
                registry[method].append(queue)
            else:
                registry[method] = [queue]
    return registry 

def queues_workers_one_step(x):
    worker_id = x[0]
    registry = {}
    for queue in x[1]:
        if queue in registry:
            registry[queue].append(worker_id)
        else:
            registry[queue] = [worker_id]
    return registry 

def all_workers_for_queue():
    # all requests queues for "info" method
    # One queue per worker

    rr = {}
    qq = {}
    all_worker_queues = client.connector.all_queues_for_method("info")
    for worker in all_worker_queues:
        try:
            x = client.rpc_sync("info", timeout=10)
        except TimeoutError:
            x = None
        
        if x is not None:
            for method, qs in registry_one_step(x).items():
                if method not in rr:
                    rr[method] = qs
                else:
                    rr[method] += qs
            for q, ws in queues_workers_one_step(x).items():
                if q not in qq:
                    qq[q] = ws
                else:
                    qq[q] += ws
    return rr, qq
            

In [76]:
registry_one_step(x)

{'info': ['requests_fs_server_2'],
 'add': ['requests_cola_1'],
 'mul': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'lista': ['requests_cola_1'],
 'tupla': ['requests_cola_1'],
 'dic': ['requests_cola_1'],
 'sleep': ['requests_cola_1'],
 'hola': ['requests_cola_2'],
 'eval_py_function': ['requests_py_eval']}

In [77]:
queues_workers_one_step(x)

{'requests_fs_server_2': ['fs_server_2'],
 'requests_cola_1': ['fs_server_2'],
 'requests_cola_2': ['fs_server_2'],
 'requests_py_eval': ['fs_server_2']}

In [78]:
r, q = all_workers_for_queue()

2025-10-29T18:22:37.461400 Client: fs_client_2 sent request with id: fs_client_2:79 to queue: requests_fs_server_1
2025-10-29T18:22:37.661109 Client: fs_client_2 received a Single RESULT with id: fs_client_2:79


2025-10-29T18:22:37.681071 Client: fs_client_2 sent request with id: fs_client_2:80 to queue: requests_fs_server_2
2025-10-29T18:22:37.924273 Client: fs_client_2 received a Single RESULT with id: fs_client_2:80


In [79]:
r

{'info': ['requests_fs_server_1', 'requests_fs_server_2'],
 'add': ['requests_cola_1', 'requests_cola_1'],
 'mul': ['requests_cola_1', 'requests_cola_1'],
 'div': ['requests_cola_1', 'requests_cola_1'],
 'lista': ['requests_cola_1', 'requests_cola_1'],
 'tupla': ['requests_cola_1', 'requests_cola_1'],
 'dic': ['requests_cola_1', 'requests_cola_1'],
 'sleep': ['requests_cola_1', 'requests_cola_1'],
 'hola': ['requests_cola_2', 'requests_cola_2'],
 'eval_py_function': ['requests_py_eval', 'requests_py_eval']}

In [80]:
q

{'requests_fs_server_1': ['fs_server_1'],
 'requests_cola_1': ['fs_server_1', 'fs_server_2'],
 'requests_cola_2': ['fs_server_1', 'fs_server_2'],
 'requests_py_eval': ['fs_server_1', 'fs_server_2'],
 'requests_fs_server_2': ['fs_server_2']}

In [81]:
t = (1,2,3,4)

In [82]:
t[:6]

(1, 2, 3, 4)

In [83]:
t[:5]

(1, 2, 3, 4)